In [1]:
import numpy as np
import pandas as pd

In [136]:
df = pd.read_csv('Recommendation_train.csv')

In [137]:
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B2,Product_Holding_B12
0,CC264719,Male,41,14,0,C1,S3,['P8'],['P16']
1,CC209679,Female,47,14,1,C1,S2,['P3'],['P13'
2,CC209679,Female,47,14,1,C1,S2,['P3'],'P20']
3,CC319633,Female,59,14,0,C2,S2,['P00'],['P11']
4,CC231413,Female,32,16,0,C1,S2,['P6'],['P8'


In [138]:
unwanted = ['[', ']']

In [139]:
for i in range(df.shape[0]):
    for j in unwanted:
        df['Product_Holding_B12'][i] = df['Product_Holding_B12'][i].replace(j, '') 

<ipython-input-139-de68ac15fd20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Product_Holding_B12'][i] = df['Product_Holding_B12'][i].replace(j, '')


In [140]:
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B2,Product_Holding_B12
0,CC264719,Male,41,14,0,C1,S3,['P8'],'P16'
1,CC209679,Female,47,14,1,C1,S2,['P3'],'P13'
2,CC209679,Female,47,14,1,C1,S2,['P3'],'P20'
3,CC319633,Female,59,14,0,C2,S2,['P00'],'P11'
4,CC231413,Female,32,16,0,C1,S2,['P6'],'P8'


In [141]:
df.nunique()

Customer_ID            37748
Gender                     2
Age                       36
Vintage                   64
Is_Active                  2
City_Category              2
Customer_Category          3
Product_Holding_B2       495
Product_Holding_B12       38
dtype: int64

In [142]:
df.shape

(63577, 9)

In [143]:
df.isnull().sum()

Customer_ID            0
Gender                 0
Age                    0
Vintage                0
Is_Active              0
City_Category          0
Customer_Category      0
Product_Holding_B2     0
Product_Holding_B12    0
dtype: int64

In [144]:
object = [feature for feature in df.columns if df[feature].dtypes == 'O' and feature not in ['Customer_ID', 'Product_Holding_B2', 'Product_Holding_B12']]

In [145]:
object

['Gender', 'City_Category', 'Customer_Category']

In [146]:
from sklearn.preprocessing import OneHotEncoder

In [147]:
ohe = OneHotEncoder()

In [148]:
final_encoded = []
for value in object:
    encoded = ohe.fit_transform(df[value].values.reshape(-1,1)).toarray()
    n = df[value].nunique()
    cols = ['{}_{}'.format(value, n) for n in range(1, n+1)]
    encoded_df = pd.DataFrame(encoded, columns = cols)
    encoded_df.index = df.index
    final_encoded.append(encoded_df)

df_final = pd.concat([df, *final_encoded], axis = 1)

In [149]:
df_final.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B2,Product_Holding_B12,Gender_1,Gender_2,City_Category_1,City_Category_2,Customer_Category_1,Customer_Category_2,Customer_Category_3
0,CC264719,Male,41,14,0,C1,S3,['P8'],'P16',0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,CC209679,Female,47,14,1,C1,S2,['P3'],'P13',1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,CC209679,Female,47,14,1,C1,S2,['P3'],'P20',1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,CC319633,Female,59,14,0,C2,S2,['P00'],'P11',1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,CC231413,Female,32,16,0,C1,S2,['P6'],'P8',1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [150]:
df_final.drop(['Gender_1', 'City_Category_1', 'Customer_Category_1'], axis = 1, inplace = True)

In [151]:
df_final.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B2,Product_Holding_B12,Gender_2,City_Category_2,Customer_Category_2,Customer_Category_3
0,CC264719,Male,41,14,0,C1,S3,['P8'],'P16',1.0,0.0,0.0,1.0
1,CC209679,Female,47,14,1,C1,S2,['P3'],'P13',0.0,0.0,1.0,0.0
2,CC209679,Female,47,14,1,C1,S2,['P3'],'P20',0.0,0.0,1.0,0.0
3,CC319633,Female,59,14,0,C2,S2,['P00'],'P11',0.0,1.0,1.0,0.0
4,CC231413,Female,32,16,0,C1,S2,['P6'],'P8',0.0,0.0,1.0,0.0


In [152]:
df_final.drop(object,axis = 1, inplace = True)

In [153]:
df1 = df_final.groupby(by = ['Product_Holding_B12'])['Customer_ID'].count().reset_index().rename(columns = {'Customer_ID':'Total'})

In [154]:
df1.head()

,Product_Holding_B12,Total
0,'P10',213
1,'P11',130
2,'P12',1380
3,'P13',5189
4,'P14',76


In [155]:
df_merged = df_final.merge(df1, left_on = 'Product_Holding_B12', right_on = 'Product_Holding_B12', how = 'left')

In [158]:
df_merged.head()

,Customer_ID,Age,Vintage,Is_Active,Product_Holding_B2,Product_Holding_B12,Gender_2,City_Category_2,Customer_Category_2,Customer_Category_3,Total
0,CC264719,41,14,0,['P8'],'P16',1.0,0.0,0.0,1.0,0.660788
1,CC209679,47,14,1,['P3'],'P13',0.0,0.0,1.0,0.0,1.000000
2,CC209679,47,14,1,['P3'],'P20',0.0,0.0,1.0,0.0,0.248373
3,CC319633,59,14,0,['P00'],'P11',0.0,1.0,1.0,0.0,0.153168
4,CC231413,32,16,0,['P6'],'P8',0.0,0.0,1.0,0.0,0.220890


In [157]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
df_merged['Total'] = sc.fit_transform(df_merged[['Total']])

In [159]:
df_pivot = df_merged.pivot(index = 'Product_Holding_B12', columns = 'Customer_ID', values = 'Total').fillna(0)

In [160]:
from scipy.sparse import csr_matrix
df_pivot_matrix = csr_matrix(df_pivot.values)

In [161]:
df_pivot.head(15)

Customer_ID,CC198726,CC198730,CC198733,CC198740,CC198748,CC198749,CC198751,CC198759,CC198767,CC198768,...,CC397418,CC397419,CC397421,CC397425,CC397428,CC397429,CC397433,CC397434,CC397435,CC397436
Product_Holding_B12,,,,,,,,,,,,,,,,,,,,,
'P10',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P11',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P12',0.118065,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.118065,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P13',0.444178,0.444178,0.444178,0.444178,0.444178,0.444178,0.0,0.000000,0.444178,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P14',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P15',0.000000,0.000000,0.000000,0.000000,0.028938,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P16',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P17',0.000000,0.000000,0.370377,0.370377,0.370377,0.000000,0.0,0.370377,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'P18',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

In [202]:
model_knn.fit(df_pivot_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [203]:
query = np.random.choice(df_pivot.shape[0])
print(query)

17


In [204]:
df_pivot.index[query]

"'P1'"

In [205]:
distances, indices = model_knn.kneighbors(df_pivot.iloc[query,:].values.reshape(1,-1), n_neighbors = 6)

In [206]:

for i in range(len(distances.flatten())):
    if i == 0:
        print('Recommendation for {0}:\n'.format(df_pivot.index[query]))
    else:
        print('{0}: {1} with distance of {2}'.format(i, df_pivot.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendation for 'P1':

1: 'P17' with distance of 1.0
2: 'P15' with distance of 1.0
3: 'P18' with distance of 1.0
4: 'P2' with distance of 1.0
5: 'P16' with distance of 1.0


In [207]:
indices

array([[17, 25, 23, 26, 28, 24]], dtype=int64)

In [208]:
np.max(indices)

28

In [210]:
df_pivot.index[np.max(indices)]

"'P2'"

In [178]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [187]:
item_similarity = cosine_similarity(df_pivot)

In [188]:
item_similarity[0]

array([1.        , 0.06009503, 0.18260213, 0.10082639, 0.00785965,
       0.01860717, 0.08048833, 0.01562457, 0.        , 0.01829624,
       0.02162272, 0.05369666, 0.        , 0.        , 0.04492166,
       0.0219436 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00263729, 0.        ,
       0.        , 0.        , 0.01211254, 0.02350178, 0.00668676,
       0.02921654, 0.21039761, 0.14693384])